In [0]:
### To use brainpy
### 1. run this part
# ! pip install cython
# ! git clone https://github.com/mobiusklein/brainpy.git
# % cd ./brainpy
# ! python setup.py install
# % cd ..

### 2. restart runtime

### 3. run this part
from brainpy import isotopic_variants

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pyteomics.mzml import read as mzmlRead
from sklearn.linear_model import LinearRegression
from pyteomics.mass import Composition
from brainpy import isotopic_variants
from sklearn.linear_model import RANSACRegressor
import time

import os
working_directory_path = '/content/DKM102+DKM106'

In [0]:
def parseSpectra(vendormzmlFilePath, featureDF):
    # sort feature dataframe by retention time (RT) and in ascending order
    featureDF = featureDF.sort_values('rt', ascending = True).reset_index(drop = True)
    # data structure for recording peaks to monitor
    monitorDict = {}
    finishedDict = {}
    # position pointer
    posPeakList = 0
    # loop through each scan
    for scan in mzmlRead(vendormzmlFilePath):
        # only process MS1
        if scan['ms level'] != 1:
            continue
        # feature extraction
        scanRT = scan['scanList']['scan'][0]['scan start time']
        # if new feature reached, register it
        while posPeakList < featureDF.shape[0] and scanRT > (featureDF.loc[posPeakList, 'rt']-0.5):
            # register in monitor dictionary
            monitorDict[(posPeakList, 
                         str(round(featureDF.loc[posPeakList, 'mz'], 4)),
                        featureDF.loc[posPeakList, 'charge'])] = []
            # advance position pointer
            posPeakList += 1
        # detect if any feature in monitor dictionary is finished
        # loop through each peak in monitor dictionary
        finishedList = []
        for posFeature, mzFeature, chargeFeature in monitorDict.keys():
            # if retention time end is passed
            if (featureDF.loc[posFeature, 'rt']+0.5) < scanRT:
                # register it for later removal
                finishedList.append((posFeature, mzFeature, chargeFeature))
            # else, propagate the features
            else:
                # add new feature to monitor list
                # first, get relevant peaks
                intArray = scan['intensity array']
                mzArray = scan['m/z array']
                targetMz = featureDF.loc[posFeature, 'mz']
                targetCharge = featureDF.loc[posFeature, 'charge']
                mzLowerBound = targetMz-3
                mzUpperBound = targetMz+3
                try:
                    indexPeakStart = min([i for i, mz in enumerate(mzArray) if mz > mzLowerBound])
                    indexPeakEnd = max([i for i, mz in enumerate(mzArray) if mz < mzUpperBound])
                except:
                    continue
                # debug
                if indexPeakStart > indexPeakEnd:
                    continue
                intArrayRelevant = intArray[indexPeakStart:indexPeakEnd+1]
                mzArrayRelevant = mzArray[indexPeakStart:indexPeakEnd+1]
                # second, register it
                monitorDict[(posFeature, mzFeature, chargeFeature)].append(\
                    (list(mzArrayRelevant), list(intArrayRelevant), scanRT))
        # move finished feature to another dictionary
        for key in finishedList:
            if len(monitorDict[key]) > 0:
                finishedDict[key] = monitorDict[key]
            del monitorDict[key]
    # final process: move all remaining feature into finished dictionary
    for key in monitorDict.keys():
        finishedDict[key] = monitorDict[key]
    # return
    return finishedDict

In [0]:
def separateSignalNoise(finishedDict, filterOut = True):
    # separate signal and noise and discard signal spectrum with fewer than 3 peaks
    separatedDict = {}
    for key in finishedDict.keys():
        # initialize empty containers
        signalList = []
        noiseList = []
        # get current spectrum list
        spectrumList = finishedDict[key]
        # process each spectrum in the list
        for spectrum in spectrumList:
            # separate signal and noise
            signalSpectrum, noiseSpectrum = separateSignalNoiseFunc(key, spectrum)
            # keep those pairs whose signal has more than 1 peaks
            if len(signalSpectrum[0]) >= 1:
                signalList.append(signalSpectrum)
                noiseList.append(noiseSpectrum)
        # register
        if filterOut == False:
            separatedDict[key] = {'signal':signalList, 'noise':noiseList}
        elif len(signalList) > 0:
            separatedDict[key] = {'signal':signalList, 'noise':noiseList}
    # return
    return separatedDict

def separateSignalNoiseFunc(masterKey, spectrum):
    # findInPhasePeaks(keyMz, PDMode, mzArray)
    # noise is defined as out-of-phase peaks
    # anything that is in-phase should be considered as signal
    # even if it is of low intensity, unexpected, ..., etc.
    indexSignal = findInPhasePeaks(masterKey[1], 1./float(masterKey[2]), spectrum[0])
    indexNoise = list(set(range(len(spectrum[0]))).difference(set(indexSignal)))
    # separate signal from noise
    mzSignalArray = []
    intSignalArray = []
    mzNoiseArray = []
    intNoiseArray = []
    for index in indexSignal:
        mzSignalArray.append(spectrum[0][index])
        intSignalArray.append(spectrum[1][index])
    for index in indexNoise:
        mzNoiseArray.append(spectrum[0][index])
        intNoiseArray.append(spectrum[1][index])
    # coerse to numpy array
    mzSignalArray = np.array(mzSignalArray)
    intSignalArray = np.array(intSignalArray)
    mzNoiseArray = np.array(mzNoiseArray)
    intNoiseArray = np.array(intNoiseArray)
    return (mzSignalArray, intSignalArray, spectrum[2]),\
        (mzNoiseArray, intNoiseArray, spectrum[2])

def findInPhasePeaks(keyMz, PDMode, mzArray):
    # convert string to float
    keyMz = float(keyMz)
    # find only consecutive inphase peaks
    # conversion
    mzArray = np.array(mzArray)
    inPhasePeakList = []
    # register keyMz
    minDiffIndex = np.argmin(np.abs(mzArray-keyMz))
    if not withinTol(mzArray[minDiffIndex], keyMz):
        return []
    inPhasePeakList.append(minDiffIndex)
    # search forward
    distInt = 1
    newMz = keyMz-distInt*PDMode
    while newMz >= np.min(mzArray):
        minDiffIndex = np.argmin(np.abs(mzArray-newMz))
        if withinTol(mzArray[minDiffIndex], newMz):
            inPhasePeakList.append(minDiffIndex)
        else:
            break
        distInt += 1
        newMz = keyMz-distInt*PDMode
    # search backward
    distInt = 1
    newMz = keyMz+distInt*PDMode
    while newMz <= np.max(mzArray):
        minDiffIndex = np.argmin(np.abs(mzArray-newMz))
        if withinTol(mzArray[minDiffIndex], newMz):
            inPhasePeakList.append(minDiffIndex)
        distInt += 1
        newMz = keyMz+distInt*PDMode
    # return
    return sorted(inPhasePeakList)

def withinTol(a, b):
    c = np.mean([a, b])
    diff = np.abs(a-b)
    errorPPM = float(diff)/float(c)*1e6
    if errorPPM > 20:
        return False
    else:
        return True

In [0]:
def quickBrainpyRegularFit(targetSpectrum, seq, charge, probeType):
    # get composition from sequence
    c = Composition(sequence = seq.upper())
    # get peak from brainpy
    mzList = []
    intList = []
    for peak in isotopic_variants(c, charge = charge):
        mzList.append(peak.mz)
        intList.append(peak.intensity)
    # sort peak
    mzArray = np.array(mzList)
    intArray = np.array(intList)
    orderKey = np.argsort(mzArray)
    mzArray = mzArray[orderKey]
    intArray = intArray[orderKey]
    # filter
    indexKeep = intArray >= 0.01*np.max(intArray)
    mzArray = mzArray[indexKeep]
    intArray = intArray[indexKeep]
    # add modification mass
    mzArray += seq.count('c')*57.02146/charge # carbamidomethyl Cys
    mzArray += 124.0838/charge # isotag C13
    if probeType == 'CBPA':
        mzArray += 77.0265/charge # account for CBA/CBPA mass difference
    # scale
    intArray *= np.max(targetSpectrum[1])/np.max(intArray)
    # return
    return (mzArray, intArray)

In [0]:
def wrapperFunc(allDF, CBAmzmlPath, CBPAmzmlPath):
    ### MS2 RT approach
    DF = allDF.loc[(allDF['Confidence'] == 'High')\
                      & (allDF['Modifications'].apply(lambda s: pd.notnull(s) and 'si' in s.lower()))]
    alignDF = DF.sort_values(by = 'XCorr')\
        .drop_duplicates(subset = ['Annotated Sequence', 'probe'])\
        [['Annotated Sequence', 'probe', 'RT [min]']]
    alignDF = alignDF.pivot(index = 'Annotated Sequence', columns = 'probe', values = 'RT [min]')\
        .reset_index(drop = False)
    alignDF.columns.name = None
    alignDF.columns = ['Annotated Sequence', 'CBA_RT', 'CBPA_RT']

    # find correlation between matched entries
    model = RANSACRegressor()
    trainData = alignDF.loc[alignDF[['CBA_RT', 'CBPA_RT']].notnull().sum(axis = 1) == 2,
                                   ['CBA_RT', 'CBPA_RT']].values
    model.fit(trainData[:, 0].reshape((-1,1)), trainData[:, 1].reshape((-1,1)))
    print('Rsq =', model.score(trainData[:, 0].reshape((-1,1)), trainData[:, 1].reshape((-1,1))))
    print('slope =', model.estimator_.coef_[0][0], ', intercept =', model.estimator_.intercept_[0])
    plt.scatter(trainData[model.inlier_mask_, 0], trainData[model.inlier_mask_, 1], 
                color = 'cornflowerblue', alpha = '0.2')
    plt.scatter(trainData[np.logical_not(model.inlier_mask_), 0], trainData[np.logical_not(model.inlier_mask_), 1], 
                color = 'orange', alpha = '0.2')
    plt.xlabel('CBA RT [min]')
    plt.ylabel('CBPA RT [min]')
    plt.plot([20, 120], [20, 120], 'r--')
    plt.legend(['identity line', 'data (inlier)', 'data (outlier)'])
    plt.title('CBA/CBPA alignment plot')
    plt.show()

    # impute missing value using the correlation found
    RTDF = alignDF.copy()
    for index, row in RTDF.loc[RTDF[['CBA_RT', 'CBPA_RT']].notnull().sum(axis = 1) == 1].iterrows():
        if pd.isnull(row['CBPA_RT']):
            RTDF.loc[index, 'CBPA_RT'] = model.estimator_.coef_[0][0]*row['CBA_RT']+model.estimator_.intercept_[0]
        else:
            RTDF.loc[index, 'CBA_RT'] = (row['CBPA_RT']-model.estimator_.intercept_[0])/model.estimator_.coef_[0][0]
    print('sanity check: after imputation, missing value summary')
    print(RTDF[['CBA_RT', 'CBPA_RT']].isnull().sum(axis = 1).value_counts())

    # construct m/z table, then merge them with filled RT
    massDF = allDF.loc[(allDF['Confidence'] == 'High')\
                      & (allDF['Modifications'].apply(lambda s: pd.notnull(s) and 'si' in s.lower()))]\
        .sort_values(by = 'XCorr')\
        .drop_duplicates(subset = ['Annotated Sequence', 'probe'])\
        [['Annotated Sequence', 'probe', 'Theo. MH+ [Da]']]
    massDF = massDF.pivot(index = 'Annotated Sequence', columns = 'probe', values = 'Theo. MH+ [Da]')\
        .reset_index(drop = False)
    massDF.columns.name = None
    for index, row in massDF.loc[massDF[['CBA', 'CBPA']].isnull().sum(axis = 1) == 1].iterrows():
        if pd.isnull(row['CBA']):
            massDF.loc[index, 'CBA'] = row['CBPA']-77.0265
        else:
            massDF.loc[index, 'CBPA'] = row['CBA']+77.0265
    # remove the extra proton
    massDF[['CBA', 'CBPA']] -= 1.0079
    massDF.columns = ['Annotated Sequence', 'CBA_mass', 'CBPA_mass']

    fullDF = RTDF.merge(right = massDF, how = 'left', on = 'Annotated Sequence')
    fullDF.head()
    
    # prepare array for feature extraction
    CBARecord = []
    for index, row in fullDF.iterrows():
        for charge in range(2, 7):
            mz = (row['CBA_mass']+charge*1.0079)/charge
            CBARecord.append([mz, charge, row['CBA_RT'], row['Annotated Sequence']])
    CBADF = pd.DataFrame.from_records(CBARecord, columns = ['mz', 'charge', 'rt', 'Annotated Sequence'])
    CBADF = CBADF.sort_values('rt', ascending = True).reset_index(drop = True)

    # extract cubes (3-D arrays of intensity, m/z, and time) for quantification
    CBAFinishedDict = parseSpectra(CBAmzmlPath, CBADF)

    # separate signal from noise by phase
    CBASeparatedDict = separateSignalNoise(CBAFinishedDict)

    # quantification by FMPI (First Model Peak Intensity)
    CBADF['FMPI'] = float('nan')
    CBADF['mz'] = CBADF['mz'].apply(lambda x: str(round(x, 4)))
    for index, row in CBADF.iterrows():
        targetKeyList = [k for k in CBASeparatedDict.keys() if k[1] == row['mz'] and k[2] == row['charge']]
        if len(targetKeyList) > 0:
            targetKey = targetKeyList[0]
            targetSpectrumList = CBASeparatedDict[targetKey]['signal']
            FMPIList = []
            for spectrum in targetSpectrumList:
                quickFit = quickBrainpyRegularFit(spectrum, row['Annotated Sequence'], row['charge'], 'CBA')
                FMPIList.append(quickFit[1][0])
            CBADF.loc[index, 'FMPI'] = max(FMPIList)

    # sanity check
    CBADF = CBADF.loc[CBADF['FMPI'].notnull()]
    print('CBA has', CBADF.shape[0], 'rows left')
    
    # prepare array for feature extraction
    CBPARecord = []
    for index, row in fullDF.iterrows():
        for charge in range(2, 7):
            mz = (row['CBPA_mass']+charge*1.0079)/charge
            CBPARecord.append([mz, charge, row['CBPA_RT'], row['Annotated Sequence']])
    CBPADF = pd.DataFrame.from_records(CBPARecord, columns = ['mz', 'charge', 'rt', 'Annotated Sequence'])
    CBPADF = CBPADF.sort_values('rt', ascending = True).reset_index(drop = True)

    # extract cubes (3-D arrays of intensity, m/z, and time) for quantification
    CBPAFinishedDict = parseSpectra(CBPAmzmlPath, CBPADF)

    # separate signal from noise by phase
    CBPASeparatedDict = separateSignalNoise(CBPAFinishedDict)

    # quantification by FMPI (First Model Peak Intensity)
    CBPADF['FMPI'] = float('nan')
    CBPADF['mz'] = CBPADF['mz'].apply(lambda x: str(round(x, 4)))
    for index, row in CBPADF.iterrows():
        targetKeyList = [k for k in CBPASeparatedDict.keys() if k[1] == row['mz'] and k[2] == row['charge']]
        if len(targetKeyList) > 0:
            targetKey = targetKeyList[0]
            targetSpectrumList = CBPASeparatedDict[targetKey]['signal']
            FMPIList = []
            for spectrum in targetSpectrumList:
                quickFit = quickBrainpyRegularFit(spectrum, row['Annotated Sequence'], row['charge'], 'CBPA')
                FMPIList.append(quickFit[1][0])
            CBPADF.loc[index, 'FMPI'] = max(FMPIList)

    # sanity check
    CBPADF = CBPADF.loc[CBPADF['FMPI'].notnull()]
    print('CBPA has', CBPADF.shape[0], 'rows left')
    
    # in-house quantification
    tmpDF1 = CBADF.groupby('Annotated Sequence')['FMPI'].sum().reset_index()
    tmpDF1.columns = ['Annotated Sequence', 'FMPI_CBA']
    tmpDF2 = CBPADF.groupby('Annotated Sequence')['FMPI'].sum().reset_index()
    tmpDF2.columns = ['Annotated Sequence', 'FMPI_CBPA']
    resultDF = tmpDF1.merge(tmpDF2, on = 'Annotated Sequence', how = 'outer')
    resultBothDF = resultDF.loc[resultDF.isnull().sum(axis = 1) == 0]
    print('in-house quantification NaN count')
    print(resultDF.isnull().sum())

    # PD quantification
    DF = allDF.loc[(allDF['Confidence'] == 'High')\
                      & (allDF['Modifications'].apply(lambda s: pd.notnull(s) and 'si' in s.lower()))]
    PADF = DF.sort_values(by = 'XCorr')\
        .drop_duplicates(subset = ['Annotated Sequence', 'probe'])\
        [['Annotated Sequence', 'Theo. MH+ [Da]', 'Charge', 'Precursor Abundance', 'probe']]
    refDF = PADF.groupby(['probe', 'Annotated Sequence'])['Precursor Abundance'].sum()\
        .reset_index().pivot(index = 'Annotated Sequence', columns = 'probe', values = 'Precursor Abundance')\
        .reset_index()
    refDF.columns.name = None
    refBothDF = refDF.loc[refDF.isnull().sum(axis = 1) == 0]
    print('PD quantification NaN count')
    print(refDF.isnull().sum())

    # plot side-by-side for comparison
    fig, ax = plt.subplots(ncols = 3, nrows = 1, figsize = (15, 5))

    # reference: PD quantification
    ax[0].scatter(refBothDF['CBA'].apply(np.log10), 
                refBothDF['CBPA'].apply(np.log10),
               color = 'cornflowerblue', alpha = 0.2)
    ax[0].plot([3, 8], [3, 8], 'r--')
    ax[0].legend(['identity line', 'data'], loc = 'lower right')
    ax[0].set_xlabel('CBA log10(abundance)')
    ax[0].set_ylabel('CBPA log10(abundance)')
    ax[0].set_title('PD quantification, n = '+str(refBothDF.shape[0]))

    # in-house quantification
    ax[1].scatter(resultBothDF['FMPI_CBA'].apply(np.log10), 
                resultBothDF['FMPI_CBPA'].apply(np.log10),
               color = 'cornflowerblue', alpha = 0.2)
    ax[1].plot([3, 8], [3, 8], 'r--')
    ax[1].legend(['identity line', 'data'], loc = 'lower right')
    ax[1].set_xlabel('CBA log10(abundance)')
    ax[1].set_ylabel('CBPA log10(abundance)')
    ax[1].set_title('In-house quantification, n = '+str(resultBothDF.shape[0]))
    
    # plot
    MVDF = alignDF[['CBA_RT', 'CBPA_RT']].isnull().sum(axis = 1).value_counts()
    barPos = np.arange(1)
    barWidth = 0.35
    ax[2].bar(x = barPos, height = [MVDF[0]], width = barWidth, bottom = 0)
    ax[2].bar(x = barPos, height = [MVDF[1]], width = barWidth, bottom = [MVDF[0]])
    ax[2].legend(['0 missing value', '1 missing value', '2 missing value'])
    ax[2].set_title('missing value counts using MS2 RT')
    ax[2].set_ylabel('count')
    ax[2].set_xlim([-1, 1])
    ax[2].set_xticklabels([])

    # adjustment
    fig.tight_layout()
    plt.savefig('download.png', dpi = 600)
    plt.show()
    
    return resultDF

In [0]:
resultDF.to_csv(os.path.join(working_directory_path, 'DKM102_HPG_CF_quant.csv'))